<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#享元模式:-一个简单的例子" data-toc-modified-id="享元模式:-一个简单的例子-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>享元模式: 一个简单的例子</a></span></li></ul></li></ul></li><li><span><a href="#Flyweight-Pattern" data-toc-modified-id="Flyweight-Pattern-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flyweight Pattern</a></span></li></ul></div>

###### 享元模式: 一个简单的例子

OOP编程中容易出现对象创建带来的性能和内存占用问题，需要满足以下条件：

1. 需要使用大量属性(可用 \_\_slots\_\_ 节省内存占用)
2. 对象太多, 难以存储或解析
3. 对象识别不是特别重要

经常使用对象池技术来实现共享对象，比如数据库中经常使用连接池来减少开销，预先建立一些连接池，每次取一个连接和数据库交互。

In [1]:
# 使用对象池技术实现享元模式

import random
from enum import Enum
TreeType = Enum('TreeType', 'apple_tree cherry_tree peach_tree')

class Tree(object):
    
    pool = dict()
    
    def __new__(cls, tree_type):
        
        obj = cls.pool.get(tree_type, None)
        if obj is None:
            obj = super().__new__(cls)
            cls.pool[tree_type] = obj
            obj.tree_type = tree_type
        return obj
    
    def render(self, age, x, y):
        
        print(f'Render a tree of type {self.tree_type} and '
              f'age {age} at ({x}, {y}).')

#### Flyweight Pattern

Designed for handling large numbers of relatively small objects, where many of the small objects are duplicates of each other.

In [2]:
class Point(object):

    __slots__ = ('x', 'y', 'z', 'color')

    def __init__(self, x=0, y=0,
                 z=0, color=None):
        self.x = x
        self.y = y
        self.z = z
        self.color = color

In [9]:
import atexit
import os
import shelve
import sys
import tempfile
import time

class Point(object):
    """Use a DBM (key–value) database that is stored in a file on disk to store its data.
    """    
    __slots__ = ()
    __dbm = shelve.open(os.path.join(tempfile.gettempdir(), 'point.db'))
    
    def __init__(self, x=0, y=0, z=0, color=None):
        
        self.x = x
        self.y = y
        self.z = z
        self.color = color
    
    def __key(self, name):
        
        return '{:X} : {}'.format(id(self), name)
    
    #下面是重点
    def __getattr__(self, name):
        
        return self.__class__.__dbm[self.__key(name)]
    
    def __setattr__(self, name, value):
        
        self.__class__.__dbm[self.__key(name)] = value
    
    atexit.register(__dbm.close)